In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Introducció
Comentari origen del dataset i objectiu del mateix. Quina pregunta intenta respondre.

In [2]:
# Read csv file
raw_df = pd.read_csv('neveras.csv', decimal=",")

In [3]:
#Rows and columns info
print(f'El data frame conté {len(raw_df.index)} mostres.')
print(f'Cada mostra disposa de {len(raw_df.columns)} atributs')

El data frame conté 488 mostres.
Cada mostra disposa de 136 atributs


Disposem de 136 variables, aquest és un nombre considerable de variables i per tant sera inevitable prescindir de moltes d'elles. En el que segeix de treball seleccionarem les variables que aporten més informació i les que puguin ser d'utilitat per tal de transformar el dataset cru del que partim en un dataset net llest per a la creacció de models. 

In [4]:
# Var list and missig values
missing_values = raw_df.isna().sum()
missing_values.sort_values()

product-name                                0
product-type                                0
price                                       0
ref                                         0
marca                                       0
val-points                                  0
val-quantity                                0
product-sending-value                       0
date                                        0
Garantía                                    5
Alturaexacta                                5
Anchuraexacta                               5
Númerodepuertas                             8
Tipodeinstalación                           9
Alimentación                               10
ConsumoAnualdeEnergía                      12
Altura                                     12
Capacidadfrigorífico                       12
Anchura                                    13
BaldasenRefrigerador                       16
EficienciaNueva                            22
Profundidad                       

Moltes variables contenen força valors nuls. Utilitzarem el nombre de valors nuls com a criteri per realitzar una subselecció de variables. Treballarem sobre les variables que contenen 25 o menys valor nuls. Aquest valor representa aproximadament un 5% de les mostres(488).

# Selecció de variables i adecució de format

In [5]:
# df per guardar les variables seleccionades
selected_df = pd.DataFrame()

## Variables descartades

In [6]:
# Mostrem els primer valors
raw_df[['product-name','ref']].head()

,product-name,ref
0,TEKA NFL 320 C INOX - Frigorífico Combi No Frost,115216026
1,INDESIT TAA 5 1 BLANCO - Frigorífico Dos Puert...,14760137
2,INDESIT LI8 SN2E X Inox - Frigorífico Combi No...,14762233
3,TEKA NFL-320 Blanco - Frigorífico Combi No Frost,115214386
4,KROMSLINE KC-185-DW BLANCO - Frigorífico Combi...,702139949


La variable product-name conte el nom del producte, de forma semblan la variable ref conté un valor numeric que relaciona les caracteristiques amb el producte. Aquestes dues variables no aporten informació útil a l'hora de crear un model.

In [7]:
# Mostrem els valors únics
print(raw_df['product-type'].unique())
print(raw_df['date'].unique())
print(raw_df['Alimentación'].unique())

['frigorificos']
['2023-04-24']
['230V' nan ' 230V' '230V ']


Les variables product-type i date contenen un mateix valor per a totes mostres, es per aquest motiu qeu no podran aportar informacó a en un futur model. Succeeix el mateix en el cas de la variable alimentació, totes els electrodomestics extrets d'aquesta font son per al mercat europeu i funcionen a una font d'alimentació de 230V.

In [8]:
quantity = raw_df['val-quantity'].str.replace('(', '').str.replace(')', '').astype('Int64')
points = raw_df['val-points']

Les variables val-quantity i val-points estan estricatament relacionades, la variables val-quantity indica el nombre de valoracions total de cada producte, metre que val-points és la mitjana de les valoracions.

In [9]:
quantity.value_counts()

0     428
1      23
2       9
6       5
3       5
4       5
10      3
8       2
5       2
7       2
20      1
12      1
13      1
14      1
Name: val-quantity, dtype: Int64

Hi ha 428 items que no tenen cap valoració. Aquestes variables poden ser molts útils per a la creació d'un model ja que aporten l'opinió del clients sobre els productes. En aquest cas, degut a la manca generalitzada de valoracions considerem que no aporten sufient infurmació per seleccionar-les.

In [10]:
raw_df[['Alturaexacta','Altura']].head()

,Alturaexacta,Altura
0,188 Centímetros,181 a 190
1,180 Centímetros,176 a 180 cm
2,189 Centímetros,181 a 190
3,188 Centímetros,181 a 190
4,185 Centímetros,181 a 190


A priori, les variables Alturaexacta i Altura representen la meteixa informació, amb la diferencia que una és una variable númerica i l'altre categorica. Per tal d'assegurar-nos calculem a continuació la correlació entre ambdues:

In [11]:
aux_df = pd.DataFrame()
aux_df["Altura"] = pd.Categorical(raw_df["Altura"])
aux_df["Alturaexacta"] = pd.Categorical(raw_df["Alturaexacta"])

corr = aux_df["Altura"].cat.codes.corr(aux_df["Alturaexacta"].cat.codes)

print(corr)

0.886325742819265


Donada l'alta correlació entre les dues variables, decidim descartar la variable altura ja que aporta una informació menys precisa i conte més valors nuls.

Es dona el mateix cas entre les variables Anchura i Anchuraexacta, per tant decidim descartar la variable Anchura.

## Variables seleccionades i adequedes

In [12]:
# Mostrem els primer valors
raw_df[['price','marca','product-sending-value','Garantía','Alturaexacta','Anchuraexacta','Númerodepuertas','Tipodeinstalación','ConsumoAnualdeEnergía','Capacidadfrigorífico','BaldasenRefrigerador','EficienciaNueva','Profundidad','Ruido','Refrigeración','Color/material']].head()

,price,marca,product-sending-value,Garantía,Alturaexacta,Anchuraexacta,Númerodepuertas,Tipodeinstalación,ConsumoAnualdeEnergía,Capacidadfrigorífico,BaldasenRefrigerador,EficienciaNueva,Profundidad,Ruido,Refrigeración,Color/material
0,"418,49 €",TEKA,"37,00 €",3 Años,188 Centímetros,60 Centímetros,2 Puertas,Libre,293 Kwh/Año,224 Litros,4 Baldas,F,64 Centímetros,41 Decibelios,No Frost,Inox
1,"393,49 €",INDESIT,"37,00 €",3 Años,180 Centímetros,70 Centímetros,2 Puertas,Libre,282 Kwh/Año,333 Litros,5 Baldas,F,69 Centímetros,44 Decibelios,Cíclico,Blanco
2,"437,49 €",INDESIT,"37,00 €",3 Años,189 Centímetros,60 Centímetros,2 Puertas,Libre,247 Kwh/Año,230 Litros,4 Baldas,E,66 Centímetros,40 Decibelios,Cíclico,Inox
3,"406,49 €",TEKA,"37,00 €",3 Años,188 Centímetros,60 Centímetros,2 Puertas,Libre,293 Kwh/Año,224 Litros,NaN,F,64 Centímetros,41 Decibelios,No Frost,Blanco
4,"455,20 €",KROMSLINE,"37,00 €",5 Años,185 Centímetros,60 Centímetros,2 Puertas,Libre,256 Kwh/Año,211 Litros,4 Baldas,E,66 Centímetros,41 Decibelios,No Frost,Blanco


In [121]:
# Seleccionem i addecuem el format
selected_df['precio'] = raw_df['price'].str.replace('.', '').str.replace(',', '.').str.replace(' €', '').astype(float)
selected_df['marca'] = raw_df['marca']
selected_df['gastos_envio'] = raw_df['product-sending-value'].str.replace('.', '').str.replace(',', '.').str.replace(' €', '').astype(float)
selected_df['garantía'] = raw_df['Garantía'].str.replace(' Años', '').astype('Int64')
selected_df['altura'] = raw_df['Alturaexacta'].str.replace('.', '').str.replace(' Centímetros', '').astype('Int64')
selected_df['anchura'] = raw_df['Anchuraexacta'].str.replace('.', '').str.replace(' Centímetros', '').astype('Int64')
selected_df['profundidad'] = raw_df['Profundidad'].str.replace(' Centímetros', '').astype('Int64')
selected_df['capacidad'] = raw_df['Capacidadfrigorífico'].str.replace(' Litros', '').str.replace('\\r', '').astype('Int64')
selected_df['puertas'] = raw_df['Númerodepuertas'].str.replace(" Puertas", "").astype('Int64')
selected_df['baldas'] = raw_df['BaldasenRefrigerador'].str.replace(' Baldas', '').str.replace(' ', '').astype('Int64')
selected_df['instalación'] = raw_df['Tipodeinstalación'].str.replace(' ', '')
selected_df['consumo_anual'] = raw_df['ConsumoAnualdeEnergía'].str.replace(' Kwh/Año', '').astype('Int64')
selected_df['eficiencia'] = raw_df['EficienciaNueva'].str.replace(' ', '')
selected_df['ruido'] = raw_df['Ruido'].str.replace(' Decibelios', '').astype('Int64')
selected_df['refrigeración'] = raw_df['Refrigeración'].str.replace(" ", "")
selected_df['color'] = raw_df['Color/material'].str.replace(" ", "")

In [122]:
# Mostrem els primer valors
selected_df.head()

,precio,marca,gastos_envio,garantía,altura,anchura,profundidad,capacidad,puertas,baldas,instalación,consumo_anual,eficiencia,ruido,refrigeración,color
0,418.49,TEKA,37.0,3,188,60,64,224,2,4,Libre,293,F,41,NoFrost,Inox
1,393.49,INDESIT,37.0,3,180,70,69,333,2,5,Libre,282,F,44,Cíclico,Blanco
2,437.49,INDESIT,37.0,3,189,60,66,230,2,4,Libre,247,E,40,Cíclico,Inox
3,406.49,TEKA,37.0,3,188,60,64,224,2,<NA>,Libre,293,F,41,NoFrost,Blanco
4,455.20,KROMSLINE,37.0,5,185,60,66,211,2,4,Libre,256,E,41,NoFrost,Blanco


In [123]:
selected_df.dtypes

precio           float64
marca             object
gastos_envio     float64
garantía           Int64
altura             Int64
anchura            Int64
profundidad        Int64
capacidad          Int64
puertas            Int64
baldas             Int64
instalación       object
consumo_anual      Int64
eficiencia        object
ruido              Int64
refrigeración     object
color             object
dtype: object

Comentari:

## Gestió dels valors nuls i valors extrems

Anem a visualitzar primer les diferents variables que hem seleccionat, quants valors nuls tenen i els seus valors únics per poder decidir com treballar-los individualment.

In [124]:
i = 0
for line in selected_df.isnull().sum():
    colname = selected_df.columns[i]
    print("\n", colname, "té", line, "NULLS")
    if len(selected_df[colname].unique()) <=25:
        print(selected_df[colname].value_counts())
    i += 1


 precio té 0 NULLS

 marca té 0 NULLS
SMEG                112
BOSCH                83
BALAY                66
LG                   52
TEKA                 35
ROMMER               33
INDESIT              24
KROMSLINE            20
WINIA                16
BEKO                 15
ARISTON HOTPOINT     13
SIEMENS              10
FAGOR                 4
CATA                  2
EDESA                 1
NIBELS                1
BUTSIR                1
Name: marca, dtype: int64

 gastos_envio té 0 NULLS
37.0    443
78.0     45
Name: gastos_envio, dtype: int64

 garantía té 5 NULLS
3     464
5      17
10      2
Name: garantía, dtype: Int64

 altura té 5 NULLS

 anchura té 5 NULLS

 profundidad té 23 NULLS

 capacidad té 12 NULLS

 puertas té 8 NULLS
2    355
1     96
4     24
0      3
3      2
Name: puertas, dtype: Int64

 baldas té 16 NULLS
4    177
3    132
5    126
7     16
2     11
6      9
0      1
Name: baldas, dtype: Int64

 instalación té 9 NULLS
Libre         455
Integrable     24
Name: 

Tenim 4 casos diferents:
- La variable no té valors nuls.
- La variable té un valor predominant que podem assignar per defecte. Per exemple els costos d'enviament solen ser en la majoria dels casos de 37€.
- La variable té una distribució més equilibrada entre les categories i no hi podem assignar cap valor. Un exemple seria la distribució de colors de les neveres.
- La variable és numèrica i podem fer servir estadística per trobar el seu valor. Com ara en el cas de l'alçada o amplada.

En el cas dels outliers visualitzarem els valors únics de les variables numèriques per veure els casos concrets.

In [125]:
def search_outliers(column):
    selected_df_column = selected_df[column].dropna().unique()
    z_scores = (selected_df_column - np.mean(selected_df_column)) / np.std(selected_df_column)
    outlier_threshold = 3
    outliers = selected_df_column[np.abs(z_scores) > outlier_threshold]
    return outliers.tolist()


In [126]:
for column in ["precio", "altura", "anchura", "profundidad", "capacidad", "ruido"]:
    print(column, search_outliers(column))

precio [9317.08, 4493.78, 3943.18, 4936.49]
altura [2030]
anchura [540]
profundidad [815]
capacidad []
ruido [254]


En el cas del preu, no obtenim valors gaire descabellats però anem a analitzar els altres casos individualment.

En el cas de les dimensions de les neveres veiem que en aquests casos concrets hi ha hagut un error en les unitats afegint un 0 de més. Veiem que en aquest cas en les dades originals el rang de les dimensions és correcte mentre que les mesures exactes no.

In [127]:
raw_df[["Altura", "Alturaexacta"]].loc[selected_df['altura'] >= 970]

,Altura,Alturaexacta
199,Mayor de 200 cm,2.030 Centímetros
413,50 a 100 cm,970 Centímetros


In [128]:
raw_df[["Anchura", "Anchuraexacta"]].loc[selected_df['anchura'] >= 540]

,Anchura,Anchuraexacta
447,51 a 54 cm,540 Centímetros


En el cas de la profunditat veiem que tot i que no tenim una variable categòrica per comparar, veient la resta de dimensions de la nevera podem asumir que succeeix el mateix.

In [129]:
raw_df[["Alturaexacta", "Anchuraexacta", "Profundidad"]].loc[selected_df['profundidad'] >= 815]

,Alturaexacta,Anchuraexacta,Profundidad
360,87 Centímetros,128 Centímetros,815 Centímetros


In [130]:
selected_df.loc[selected_df['altura'] >= 970, 'altura'] /= 10
selected_df.loc[selected_df['anchura'] >= 540, 'anchura'] /= 10
selected_df.loc[selected_df['profundidad'] >= 815, 'profundidad'] = np.ceil(selected_df.loc[selected_df['profundidad'] >= 815, 'profundidad'] / 10)
for column in ["altura", "anchura", "profundidad"]:
    print(column, search_outliers(column))

altura [2]
anchura [179]
profundidad [0]


In [ ]:
# Repetim per les altres columnes
raw_df['val-points'] = raw_df['val-points'].str.replace(',', '.').astype(float)
raw_df['val-quantity'] = raw_df['val-quantity'].str.replace('(', '').str.replace(')', '').astype(int)
raw_df['product-sending-value'] = raw_df['product-sending-value'].str.replace('.', '').str.replace(',', '.').str.replace(' €', '').astype(float)

In [ ]:
raw_df['Refrigeración'] = raw_df['Refrigeración'].str.replace(" ", "").fillna("NoFrost")
raw_df['Númerodepuertas'] = raw_df['Númerodepuertas'].str.replace(" Puertas", "").fillna("2").astype(int)
raw_df['Tipodeinstalación'] = raw_df['Tipodeinstalación'].str.replace(' ', '')
raw_df['Cajones/Cestas'] = raw_df['Cajones/Cestas'].str.replace(" Cajoneras", "").fillna("2").astype(int)
raw_df['Iluminación'] = raw_df['Iluminación'].fillna("Unknown")
raw_df['Color/material'] = raw_df['Color/material'].str.replace(" ", "").fillna("Unknown")
raw_df['Tirador'] = raw_df['Tirador'].str.replace(" ", "").fillna("Unknown")
raw_df['Alimentación'] = raw_df['Alimentación'].str.replace('V', '').str.replace(' ', '').fillna("230").astype(int)
raw_df['EficienciaNueva'] = raw_df['EficienciaNueva'].str.replace(' ', '').fillna("E")
raw_df['ConsumoAnualdeEnergía'] = raw_df['ConsumoAnualdeEnergía'].str.replace(' Kwh/Año', '')
average = raw_df['ConsumoAnualdeEnergía'].dropna().astype(float).mean()
raw_df['ConsumoAnualdeEnergía'] = raw_df['ConsumoAnualdeEnergía'].fillna(average).astype(int)

raw_df['Ruido'] = raw_df['Ruido'].str.replace(' Decibelios', '')
ruidocol = raw_df['Ruido'].replace("254", np.nan).dropna().astype(float)
linear_values = 10 ** (ruidocol / 10)
average_decibel = 10 * np.log10(np.mean(linear_values))
raw_df['Ruido'] = raw_df['Ruido'].fillna(average_decibel).astype(int)


raw_df['Alturaexacta'] = raw_df['Alturaexacta'].str.replace(' Centímetros', '') #falta gestionar NULLS i convertir a INT
raw_df['Anchuraexacta'] = raw_df['Anchuraexacta'].str.replace(' Centímetros', '') #falta gestionar NULLS i convertir a INT
raw_df['Profundidad'] = raw_df['Profundidad'].str.replace(' Centímetros', '') #falta gestionar NULLS i convertir a INT
raw_df['Garantía'] = raw_df['Garantía'].str.replace(' Años', '').fillna('3').astype(int)
raw_df['BaldasenRefrigerador'] = raw_df['BaldasenRefrigerador'].str.replace(' Baldas', '').str.replace(' ', '') #falta gestionar NULLS i convertir a INT
raw_df['Capacidadfrigorífico'] = raw_df['Capacidadfrigorífico'].str.replace(' Litros', '').str.replace('\\r', '') #falta gestionar NULLS i convertir a INT

i = 0
for line in raw_df.isnull().sum():
    if line < 50:
        colname = raw_df.columns[i]
        print(colname, "has", line, "NULLS")
        print(raw_df[colname].unique(), "\n")
    i += 1